In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler  

Ground features: [NETRAD,	Rg,	H,	LE,	G,	u,	Ta_max,	Ta_min,	Ta_mean,	Tc_max,	Tc_min,	Tc_mean,	VPD_max,	VPD_min,	VPD_mean,	RH_mean,	Pa]

Unused Features: DATE, YEAR

Predicting: ET

//

Data Preprocessing

In [22]:
# load dataset
df = pd.read_csv('OLA_daily_2021.csv') 
df1 = pd.read_csv('OLA_daily_2022.csv')
df2 = pd.read_csv('VAC_daily_2022.csv')
df3 = pd.read_csv('VAC_daily_2022.csv')
df4 = pd.read_csv('WWF_daily_2021.csv')
df5 = pd.read_csv('WWF_daily_2022.csv')

df = pd.concat([df, df1, df2, df3, df4, df5])

# drop unused columns
df = df.drop(columns='DATE')
df = df.drop(columns='YEAR')
df = df.dropna()

df = df[df.ET != -9999]
df = df[df.NETRAD != -9999]
df = df[df.Rg != -9999]
df = df[df.H != -9999]
df = df[df.LE != -9999]
df = df[df.G != -9999]
df = df[df.u != -9999]
df = df[df.Ta_max != -9999]
df = df[df.Ta_min != -9999]
df = df[df.Ta_mean != -9999]
df = df[df.Tc_max != -9999]
df = df[df.Tc_min != -9999]
df = df[df.Tc_mean != -9999]
df = df[df.VPD_max != -9999]
df = df[df.VPD_min != -9999]
df = df[df.VPD_mean != -9999]
df = df[df.RH_mean != -9999]
df = df[df.Pa != -9999]

# make X, y sets
y = df['ET']
X = df.drop(labels=['ET'], axis=1)
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25)

# scale features to be in [0, 1]
scaler = MinMaxScaler()

# train and fit only from training data
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Generic SciKit-Learn RandomForest Regression Model

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from numpy import mean

scores = []
for random_state in range(50):
  regr = RandomForestRegressor(max_depth=10, random_state=0, max_features='sqrt', bootstrap=True)
  regr.fit(X_train, y_train)

  #y_pred = regr.predict(X_test)
  scores.append(regr.score(X_test, y_test))
print("Averaged for 50 random seeds...")
print("Generic model R^2 score: (-inf, 1], with 1 being highest fit score possible.")
print(mean(scores))


Averaged for 50 random seeds...
Generic model R^2 score: (-inf, 1], with 1 being highest fit score possible.
0.9638641625808247


Hyperparameter Tuning: RandomSearch

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['none', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 150, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

test_regr = RandomForestRegressor()
clf = RandomizedSearchCV(estimator=test_regr, param_distributions=random_grid, n_iter=1000, cv=None, verbose=2, random_state=99)
search = clf.fit(X_train, y_train)
print(search.best_params_)
